# **LTX-2 Distilled For Text/Image to Video**

- The hidden settings for this notebook are for the distilled  model versions of LTX-2. You can generate up to 4 seconds of a 480p video with the default models and settings on the T4 GPU. If you uploaded an image but want text to video generation, then you can enable `disable_image_for_Text_to_Video`

- Github project page: https://github.com/Lightricks/LTX-2

- Models source: (1) https://huggingface.co/Kijai/LTXV2_comfy/tree/main (2) https://huggingface.co/Comfy-Org/ltx-2/tree/main/split_files/text_encoders
- Notebook source: https://github.com/Isi-dev/Google-Colab_Notebooks
- Premium notebooks: https://isinse.gumroad.com/

In [ ]:
# @markdown # 💥Prepare Environment

!pip install torch torchvision torchaudio

%cd /content
from IPython.display import clear_output
clear_output()
!pip install -q torchsde einops diffusers accelerate
!pip install av spandrel albumentations onnx opencv-python onnxruntime
!git clone  --branch ComfyUI_22_01_2026_v0.10.0 https://github.com/Isi-dev/ComfyUI.git
!pip install -r /content/ComfyUI/requirements.txt
clear_output()


%cd /content/ComfyUI/custom_nodes

import subprocess

!git clone --branch kj_1.2.6 https://github.com/Isi-dev/ComfyUI_KJNodes
!git clone --branch ComfyUI_GGUF_22_01_2026 https://github.com/Isi-dev/ComfyUI_GGUF.git

%cd /content/ComfyUI/custom_nodes/ComfyUI_KJNodes
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI_GGUF
!pip install -r requirements.txt

def install_apt_packages():
    packages = ['aria2', 'ffmpeg']

    try:
        # Run apt install silently (using -qq)
        subprocess.run(
            ['apt-get', '-y', 'install', '-qq'] + packages,
            check=True,
            capture_output=True
        )
        print("✓ apt packages installed")
    except subprocess.CalledProcessError as e:
        print(f"✗ Error installing apt packages: {e.stderr.decode().strip() or 'Unknown error'}")


print("Installing apt packages...")
install_apt_packages()

clear_output()


%cd /content/ComfyUI

# !apt -y install -qq aria2 ffmpeg

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import sys
from pathlib import Path
import random
from typing import Sequence, Mapping, Any, Union
import torch

import numpy as np
import cv2
from PIL import Image
import gc
import imageio
import shutil
from google.colab import files
from IPython.display import display, HTML, Image as IPImage
sys.path.insert(0, '/content/ComfyUI')


from nodes import NODE_CLASS_MAPPINGS
from nodes import LoraLoaderModelOnly


def download_with_aria2c(link, folder="/content/ComfyUI/models/loras"):
    import os

    filename = link.split("/")[-1]
    command = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d {folder} -o {filename}"

    print("Executing download command:")
    print(command)

    os.makedirs(folder, exist_ok=True)
    get_ipython().system(command)

    return filename



def download_civitai_model(civitai_link, civitai_token, folder="/content/ComfyUI/models/loras"):
    import os
    import time

    os.makedirs(folder, exist_ok=True)

    try:
        model_id = civitai_link.split("/models/")[1].split("?")[0]
    except IndexError:
        raise ValueError("Invalid Civitai URL format. Please use a link like: https://civitai.com/api/download/models/1523247?...")

    civitai_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor"
    if civitai_token:
        civitai_url += f"&token={civitai_token}"

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"model_{timestamp}.safetensors"

    full_path = os.path.join(folder, filename)

    download_command = f"wget --max-redirect=10 --show-progress \"{civitai_url}\" -O \"{full_path}\""
    print("Downloading from Civitai...")

    os.system(download_command)

    local_path = os.path.join(folder, filename)
    if os.path.exists(local_path) and os.path.getsize(local_path) > 0:
        print(f"LoRA downloaded successfully: {local_path}")
    else:
        print(f"❌ LoRA download failed or file is empty: {local_path}")

    return filename

def download_lora(link, folder="/content/ComfyUI/models/loras", civitai_token=None):
    """
    Download a model file, automatically detecting if it's a Civitai link or huggingface download.

    Args:
        link: The download URL (either huggingface or Civitai)
        folder: Destination folder for the download
        civitai_token: Optional token for Civitai downloads (required if link is from Civitai)

    Returns:
        The filename of the downloaded model
    """
    if "civitai.com" in link.lower():
        if not civitai_token:
            raise ValueError("Civitai token is required for Civitai downloads")
        return download_civitai_model(link, civitai_token, folder)
    else:
        return download_with_aria2c(link, folder)

def model_download(url: str, dest_dir: str, filename: str = None, silent: bool = True) -> bool:
    """
    Colab-optimized download with aria2c

    Args:
        url: Download URL
        dest_dir: Target directory (will be created if needed)
        filename: Optional output filename (defaults to URL filename)
        silent: If True, suppresses all output (except errors)

    Returns:
        bool: True if successful, False if failed
    """
    try:
        # Create destination directory
        Path(dest_dir).mkdir(parents=True, exist_ok=True)

        # Set filename if not specified
        if filename is None:
            filename = url.split('/')[-1].split('?')[0]  # Remove URL parameters

        # Build command
        cmd = [
            'aria2c',
            '--console-log-level=error',
            '-c', '-x', '16', '-s', '16', '-k', '1M',
            '-d', dest_dir,
            '-o', filename,
            url
        ]

        # Add silent flags if requested
        if silent:
            cmd.extend(['--summary-interval=0', '--quiet'])
            print(f"Downloading {filename}...", end=' ', flush=True)

        # Run download
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)

        if silent:
            print("Done!")
        else:
            print(f"Downloaded {filename} to {dest_dir}")
        return filename

    except subprocess.CalledProcessError as e:
        error = e.stderr.strip() or "Unknown error"
        print(f"\nError downloading {filename}: {error}")
        return False
    except Exception as e:
        print(f"\nError: {str(e)}")
        return False



def get_value_at_index(obj: Union[Sequence, Mapping], index: int) -> Any:
    """Returns the value at the given index of a sequence or mapping.

    If the object is a sequence (like list or string), returns the value at the given index.
    If the object is a mapping (like a dictionary), returns the value at the index-th key.

    Some return a dictionary, in these cases, we look for the "results" key

    Args:
        obj (Union[Sequence, Mapping]): The object to retrieve the value from.
        index (int): The index of the value to retrieve.

    Returns:
        Any: The value at the given index.

    Raises:
        IndexError: If the index is out of bounds for the object and the object is not a mapping.
    """
    try:
        return obj[index]
    except KeyError:
        return obj["result"][index]

def find_path(name: str, path: str = None) -> str:
    """
    Recursively looks at parent folders starting from the given path until it finds the given name.
    Returns the path as a Path object if found, or None otherwise.
    """
    # If no path is given, use the current working directory
    if path is None:
        path = os.getcwd()

    # Check if the current directory contains the name
    if name in os.listdir(path):
        path_name = os.path.join(path, name)
        print(f"{name} found: {path_name}")
        return path_name

    # Get the parent directory
    parent_directory = os.path.dirname(path)

    # If the parent directory is the same as the current directory, we've reached the root and stop the search
    if parent_directory == path:
        return None

    # Recursively call the function with the parent directory
    return find_path(name, parent_directory)

def add_comfyui_directory_to_sys_path() -> None:
    """
    Add 'ComfyUI' to the sys.path
    """
    comfyui_path = find_path("ComfyUI")
    if comfyui_path is not None and os.path.isdir(comfyui_path):
        sys.path.append(comfyui_path)
        print(f"'{comfyui_path}' added to sys.path")

def add_extra_model_paths() -> None:
    """
    Parse the optional extra_model_paths.yaml file and add the parsed paths to the sys.path.
    """
    try:
        from main import load_extra_path_config
    except ImportError:
        print(
            "Could not import load_extra_path_config from main.py. Looking in utils.extra_config instead."
        )
        from utils.extra_config import load_extra_path_config

    extra_model_paths = find_path("extra_model_paths.yaml")

    if extra_model_paths is not None:
        load_extra_path_config(extra_model_paths)
    else:
        print("Could not find the extra_model_paths config file.")


# add_comfyui_directory_to_sys_path()
# add_extra_model_paths()

import asyncio
import nest_asyncio

def import_custom_nodes() -> None:
    """Load all built-in and external custom nodes in a Jupyter/Colab-safe way."""

    from nodes import init_builtin_extra_nodes, init_external_custom_nodes

    async def loader():
        # Load built-in extra nodes
        import_failed = await init_builtin_extra_nodes()

        # Load external custom nodes
        await init_external_custom_nodes()

        if import_failed:
            print("WARNING: Some comfy_extras nodes failed to import:")
            for node in import_failed:
                print(" -", node)

    try:
        # Try running normally (works in scripts)
        asyncio.run(loader())
    except RuntimeError:
        # Already inside an event loop → Jupyter/Colab
        nest_asyncio.apply()
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(loader())


ltx_model = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/diffusion_models/ltx-2-19b-distilled_Q4_K_M.gguf" # @param {"type":"string"}
dit_model=model_download(ltx_model, "/content/ComfyUI/models/unet")

text_encoder_link = "https://huggingface.co/Comfy-Org/ltx-2/resolve/main/split_files/text_encoders/gemma_3_12B_it_fp4_mixed.safetensors" # @param {"type":"string"}
text_encoder_model = model_download(text_encoder_link, "/content/ComfyUI/models/text_encoders")

text_encoder2_link = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/text_encoders/ltx-2-19b-embeddings_connector_distill_bf16.safetensors" # @param {"type":"string"}
text_encoder2_model = model_download(text_encoder2_link, "/content/ComfyUI/models/text_encoders")

vae_link = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/VAE/LTX2_video_vae_bf16.safetensors" # @param {"type":"string"}
vae_model = model_download(vae_link, "/content/ComfyUI/models/vae")

vae_audio_link = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/VAE/LTX2_audio_vae_bf16.safetensors" # @param {"type":"string"}
vae_audio_model = model_download(vae_audio_link, "/content/ComfyUI/models/vae")

upscaler_link = "https://huggingface.co/Lightricks/LTX-2/resolve/main/ltx-2-spatial-upscaler-x2-1.0.safetensors" # @param {"type":"string"}
upscaler_model = model_download(upscaler_link, "/content/ComfyUI/models/latent_upscale_models")

download_loRA_1 = True # @param {type:"boolean"}
lora_1_download_url = "https://huggingface.co/Lightricks/LTX-2-19b-LoRA-Camera-Control-Dolly-Left/resolve/main/ltx-2-19b-lora-camera-control-dolly-left.safetensors"# @param {"type":"string"}

download_loRA_2 = False # @param {type:"boolean"}
lora_2_download_url = ""# @param {"type":"string"}

download_loRA_3 = False # @param {type:"boolean"}
lora_3_download_url = ""# @param {"type":"string"}

token_if_civitai_url = ""# @param {"type":"string"}

lora_1 = None
if download_loRA_1:
    lora_1 = download_lora(lora_1_download_url, civitai_token=token_if_civitai_url)
# Validate loRA file extension
valid_extensions = {'.safetensors', '.ckpt', '.pt', '.pth', '.sft'}
if lora_1:
    if not any(lora_1.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_1}")
        lora_1 = None
    else:
        clear_output()
        print("loRA 1 downloaded succesfully!")

lora_2 = None
if download_loRA_2:
    lora_2 = download_lora(lora_2_download_url, civitai_token=token_if_civitai_url)
if lora_2:
    if not any(lora_2.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_2}")
        lora_2 = None
    else:
        clear_output()
        print("loRA 2 downloaded succesfully!")

lora_3 = None
if download_loRA_3:
    lora_3 = download_lora(lora_3_download_url, civitai_token=token_if_civitai_url)
if lora_3:
    if not any(lora_3.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_3}")
        lora_3 = None
    else:
        clear_output()
        print("loRA 3 downloaded succesfully!")


def upload_file():
    """Handle file upload (image or video) and return paths."""
    os.makedirs('/content/ComfyUI/input', exist_ok=True)
    uploaded = files.upload()

    paths = []
    for filename in uploaded.keys():
        src_path = f'/content/ComfyUI/{filename}'
        dest_path = f'/content/ComfyUI/input/{filename}'
        shutil.move(src_path, dest_path)
        paths.append(dest_path)
        print(f"File saved to: {dest_path}")

    return paths[0] if paths else None

def upload_fileInt():
    """Handle file upload (image or video) and return paths."""
    os.makedirs('/content/ComfyUI/output', exist_ok=True)
    uploaded = files.upload()

    paths = []
    for filename in uploaded.keys():
        src_path = f'/content/ComfyUI/{filename}'
        dest_path = f'/content/ComfyUI/output/{filename}'
        shutil.move(src_path, dest_path)
        paths.append(dest_path)
        print(f"File saved to: {dest_path}")

    return paths[0] if paths else None

def extract_frames(video_path, max_frames=None):
    """Extract frames from video and return as a batch tensor."""
    vidcap = cv2.VideoCapture(video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frames = []

    while True:
        success, frame = vidcap.read()
        if not success or (max_frames and len(frames) >= max_frames):
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = torch.from_numpy(frame).float() / 255.0
        frames.append(frame)

    if not frames:
        return None, fps

    # Stack frames into a batch tensor: (N, H, W, 3)
    batch = torch.stack(frames, dim=0)
    # print(f"Extracted {len(frames)} frames (shape: {batch.shape})")
    return batch, fps


def select_every_n_frame_tensor(
    frames_tensor: torch.Tensor,
    fps: float,
    n: int,
    skip_first: int = 0,
    max_output_frames: int = 0
):
    if frames_tensor is None or frames_tensor.ndim != 4:
        raise ValueError("frames_tensor must be a 4D tensor of shape (N, H, W, C)")
    if n < 1:
        raise ValueError("n must be >= 1")

    total_frames = frames_tensor.shape[0]

    if skip_first >= total_frames:
        print("No frames available after skipping.")
        return None, 0.0

    frames_to_use = frames_tensor[skip_first:]

    # Select every nth frame
    selected_frames = frames_to_use[::n]

    # Cap output if needed
    if max_output_frames > 0 and selected_frames.shape[0] > max_output_frames:
        selected_frames = selected_frames[:max_output_frames]

    adjusted_fps = fps / n

    if max_output_frames:
        print(f"Frame cap: {max_output_frames} -> Final output: {selected_frames.shape[0]} frames")
    # print(f"Adjusted FPS: {adjusted_fps:.2f}  -> Final output: {selected_frames.shape[0]} frames")

    return selected_frames, adjusted_fps

def is_video_file(path):
    video_extensions = {".mp4", ".avi", ".mov", ".mkv", ".flv", ".wmv", ".webm"}
    return os.path.splitext(path)[1].lower() in video_extensions

def get_last_frame_tensor(video_path: str) -> torch.Tensor | None:
    """Return the last frame of a video as a tensor of shape (1, H, W, C)."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None

    # Jump directly to the last frame
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count == 0:
        return None

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count - 1)
    success, frame = cap.read()
    cap.release()

    if not success:
        return None

    # Convert BGR → RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert to float tensor in range [0,1], shape (1,H,W,C)
    tensor = torch.from_numpy(frame).float().unsqueeze(0) / 255.0
    return tensor


def swapT(pa, f, s):
    if pa == f:
        pa = s
    return pa


def image_width_height(image):
    if image.ndim == 4:
        _, height, width, _ = image.shape
    elif image.ndim == 3:
        height, width, _ = image.shape
    else:
        raise ValueError(f"Unsupported image shape: {image.shape}")
    return width, height

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    for obj in list(globals().values()):
        if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
            del obj
    gc.collect()


def upload_image():
    """Handle image upload in Colab and store in /content/ComfyUI/input/"""
    from google.colab import files
    import os
    import shutil

    os.makedirs('/content/ComfyUI/input', exist_ok=True)

    uploaded = files.upload()

    # Move each uploaded file to ComfyUI input directory
    for filename in uploaded.keys():
        src_path = f'/content/ComfyUI/{filename}'
        dest_path = f'/content/ComfyUI/input/{filename}'

        shutil.move(src_path, dest_path)
        # print(f"Image saved to: {dest_path}")
        return dest_path

    return None


output_path =""

clear_output()
print("✅ Environment Setup Complete!")


In [ ]:
# @markdown # 💥2. Upload Image (Optional)
file_uploaded = upload_file()
display_upload = False # @param {type:"boolean"}
if display_upload:
    if file_uploaded.lower().endswith(('.png', '.jpg', '.jpeg')):
        display(IPImage(filename=file_uploaded))
    else:
        display_video(file_uploaded)
# @markdown ---

In [5]:
# @markdown # 💥3. Generate Video



from comfy_api.latest import Input, Types
import folder_paths

def save_video_from_components(video, filename_prefix="video/LTX", format="auto", codec="auto"):
    width, height = video.get_dimensions()

    full_output_folder, filename, counter, _, _ = (
        folder_paths.get_save_image_path(
            filename_prefix,
            folder_paths.get_output_directory(),
            width,
            height
        )
    )

    ext = Types.VideoContainer.get_extension(format)
    path = os.path.join(full_output_folder, f"{filename}_{counter:05}_.{ext}")

    video.save_to(
        path,
        format=Types.VideoContainer(format),
        codec=codec,
        metadata=None
    )

    return path


def display_video(video_path):
    from IPython.display import HTML
    from base64 import b64encode

    video_data = open(video_path,'rb').read()

    # Determine MIME type based on file extension
    if video_path.lower().endswith('.mp4'):
        mime_type = "video/mp4"
    elif video_path.lower().endswith('.webm'):
        mime_type = "video/webm"
    elif video_path.lower().endswith('.webp'):
        mime_type = "image/webp"
    else:
        mime_type = "video/mp4"  # default

    data_url = f"data:{mime_type};base64," + b64encode(video_data).decode()

    display(HTML(f"""
    <video width=512 controls autoplay loop>
        <source src="{data_url}" type="{mime_type}">
    </video>
    """))


def mainLTX(
    image_path: str = None,
    prompt: str = "The lady says, 'Hello my friend. Welcome to LTX2. You will enjoy creating videos with audio on LTX2.'",
    width: int = 832,
    height: int = 480,
    seed: int = 10,
    frames: int = 73,
    fpsv: int = 25,
    use_lora: bool = True,
    LoRA_Strength: float = 1.00,
    use_lora2: bool = True,
    LoRA_Strength2: float = 1.00,
    use_lora3: bool = True,
    LoRA_Strength3: float = 1.00,

):
    import_custom_nodes()
    clear_output()
    with torch.inference_mode():


        # print("loading image")
        loadimage = NODE_CLASS_MAPPINGS["LoadImage"]()
        if image_path is not None:
            loadimage_98 = loadimage.load_image(image=image_path)
            image_strength = 1.0
            image_bypass = False

        else:

            batch = 1
            noise_image = torch.full((1, height, width, 3), 0.5)


            loadimage_98 = (noise_image, None)

            image_strength = 0.0
            image_bypass = True

        ksamplerselect = NODE_CLASS_MAPPINGS["KSamplerSelect"]()
        ksamplerselect_105 = ksamplerselect.EXECUTE_NORMALIZED(sampler_name="euler")

        ksamplerselect_106 = ksamplerselect.EXECUTE_NORMALIZED(
            sampler_name="gradient_estimation"
        )

        manualsigmas = NODE_CLASS_MAPPINGS["ManualSigmas"]()
        manualsigmas_107 = manualsigmas.EXECUTE_NORMALIZED(
            sigmas="0.909375, 0.725, 0.421875, 0.0"
        )

        randomnoise = NODE_CLASS_MAPPINGS["RandomNoise"]()
        randomnoise_114 = randomnoise.EXECUTE_NORMALIZED(
            noise_seed=0
        )

        primitiveint = NODE_CLASS_MAPPINGS["PrimitiveInt"]()
        primitiveint_123 = primitiveint.EXECUTE_NORMALIZED(value=frames)



        randomnoise_133 = randomnoise.EXECUTE_NORMALIZED(
            noise_seed=seed
        )

        manualsigmas_134 = manualsigmas.EXECUTE_NORMALIZED(
            sigmas="1., 0.99375, 0.9875, 0.98125, 0.975, 0.909375, 0.725, 0.421875, 0.0"
        )



        conditioningzeroout = NODE_CLASS_MAPPINGS["ConditioningZeroOut"]()
        ltxvconditioning = NODE_CLASS_MAPPINGS["LTXVConditioning"]()
        cfgguider = NODE_CLASS_MAPPINGS["CFGGuider"]()
        resizeimagemasknode = NODE_CLASS_MAPPINGS["ResizeImageMaskNode"]()
        resizeimagesbylongeredge = NODE_CLASS_MAPPINGS["ResizeImagesByLongerEdge"]()
        ltxvpreprocess = NODE_CLASS_MAPPINGS["LTXVPreprocess"]()
        getimagesize = NODE_CLASS_MAPPINGS["GetImageSize"]()
        emptyimage = NODE_CLASS_MAPPINGS["EmptyImage"]()
        imagescaleby = NODE_CLASS_MAPPINGS["ImageScaleBy"]()
        emptyltxvlatentvideo = NODE_CLASS_MAPPINGS["EmptyLTXVLatentVideo"]()
        ltxvimgtovideoinplace = NODE_CLASS_MAPPINGS["LTXVImgToVideoInplace"]()
        ltxvemptylatentaudio = NODE_CLASS_MAPPINGS["LTXVEmptyLatentAudio"]()
        ltxvconcatavlatent = NODE_CLASS_MAPPINGS["LTXVConcatAVLatent"]()
        samplercustomadvanced = NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
        ltxvseparateavlatent = NODE_CLASS_MAPPINGS["LTXVSeparateAVLatent"]()
        ltxvlatentupsampler = NODE_CLASS_MAPPINGS["LTXVLatentUpsampler"]()
        ltxvcropguides = NODE_CLASS_MAPPINGS["LTXVCropGuides"]()
        vaedecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
        ltxvaudiovaedecode = NODE_CLASS_MAPPINGS["LTXVAudioVAEDecode"]()
        createvideo = NODE_CLASS_MAPPINGS["CreateVideo"]()
        # savevideo = NODE_CLASS_MAPPINGS["SaveVideo"]()

        for q in range(1):


            resizeimagemasknode_102 = resizeimagemasknode.EXECUTE_NORMALIZED(
                input=get_value_at_index(loadimage_98, 0),
                scale_method="lanczos",
                resize_type={
                    "resize_type": "scale dimensions",
                    "width": width,
                    "height": height,
                    "crop": "center",
                }
            )

            resizeimagesbylongeredge_140 = resizeimagesbylongeredge.EXECUTE_NORMALIZED(
                longer_edge=848, images=get_value_at_index(resizeimagemasknode_102, 0)
            )

            ltxvpreprocess_126 = ltxvpreprocess.EXECUTE_NORMALIZED(
                img_compression=33,
                image=get_value_at_index(resizeimagesbylongeredge_140, 0),
            )

            getimagesize_125 = getimagesize.EXECUTE_NORMALIZED(
                image=get_value_at_index(resizeimagemasknode_102, 0),

            )

            emptyimage_112 = emptyimage.generate(
                width=get_value_at_index(getimagesize_125, 0),
                height=get_value_at_index(getimagesize_125, 1),
                batch_size=1,
                color=0,
            )


            imagescaleby_122 = imagescaleby.upscale(
                upscale_method="lanczos",
                scale_by=0.5,
                image=get_value_at_index(emptyimage_112, 0),
            )

            getimagesize_124 = getimagesize.EXECUTE_NORMALIZED(
                image=get_value_at_index(imagescaleby_122, 0),

            )


            emptyltxvlatentvideo_127 = emptyltxvlatentvideo.EXECUTE_NORMALIZED(
                width=get_value_at_index(getimagesize_124, 0),
                height=get_value_at_index(getimagesize_124, 1),
                length=get_value_at_index(primitiveint_123, 0),
                batch_size=1,
            )


            print("loading text encoder")
            dualcliploader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
            dualcliploader_145 = dualcliploader.load_clip(
                clip_name1=text_encoder_model,
                clip_name2=text_encoder2_model,
                type="ltxv",
                device="default",
            )



            cliptextencode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
            cliptextencode_131 = cliptextencode.encode(
                text=prompt,
                clip=get_value_at_index(dualcliploader_145, 0),
            )

            # print("deleting dual clip loader")
            del dualcliploader_145
            torch.cuda.empty_cache()
            gc.collect()

            clear_output()

            conditioningzeroout_132 = conditioningzeroout.zero_out(
                conditioning=get_value_at_index(cliptextencode_131, 0)
            )

            ltxvconditioning_111 = ltxvconditioning.EXECUTE_NORMALIZED(
                frame_rate=25,
                positive=get_value_at_index(cliptextencode_131, 0),
                negative=get_value_at_index(conditioningzeroout_132, 0),
            )


            # print("loading vae")
            vaeloader = NODE_CLASS_MAPPINGS["VAELoader"]()
            vaeloader_144 = vaeloader.load_vae(vae_name=vae_model)

            ltxvimgtovideoinplace_128 = ltxvimgtovideoinplace.EXECUTE_NORMALIZED(
                strength=image_strength,
                bypass=image_bypass,
                vae=get_value_at_index(vaeloader_144, 0),
                image=get_value_at_index(ltxvpreprocess_126, 0),
                latent=get_value_at_index(emptyltxvlatentvideo_127, 0),
            )

            # print("deleting vae")
            del vaeloader_144
            torch.cuda.empty_cache()
            gc.collect()

            # print("loading audio vae")
            vaeloaderkj = NODE_CLASS_MAPPINGS["VAELoaderKJ"]()
            vaeloaderkj_142 = vaeloaderkj.load_vae(
                vae_name=vae_audio_model,
                device="main_device",
                weight_dtype="fp16",
            )

            ltxvemptylatentaudio_110 = ltxvemptylatentaudio.EXECUTE_NORMALIZED(
                frames_number=get_value_at_index(primitiveint_123, 0),
                frame_rate=fpsv,
                batch_size=1,
                audio_vae=get_value_at_index(vaeloaderkj_142, 0),
            )


            if image_path is not None:
                ltxvconcatavlatent_121 = ltxvconcatavlatent.EXECUTE_NORMALIZED(
                    video_latent=get_value_at_index(ltxvimgtovideoinplace_128, 0),
                    audio_latent=get_value_at_index(ltxvemptylatentaudio_110, 0),
                )

            else:
                ltxvconcatavlatent_121 = ltxvconcatavlatent.EXECUTE_NORMALIZED(
                    video_latent=get_value_at_index(emptyltxvlatentvideo_127, 0),
                    audio_latent=get_value_at_index(ltxvemptylatentaudio_110, 0),
                )


            print("loading unet")
            unetloadergguf = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
            unetloadergguf_146 = unetloadergguf.load_unet(
                unet_name="ltx-2-19b-distilled_Q4_K_M.gguf"
            )

            unetloadergguf_146 = get_value_at_index(unetloadergguf_146, 0)

            load_lora = LoraLoaderModelOnly()
            load_lora2 = LoraLoaderModelOnly()
            load_lora3 = LoraLoaderModelOnly()

            if use_lora and lora_1 is not None:
                print("Loading LoRA...")
                unetloadergguf_146 = load_lora.load_lora_model_only(unetloadergguf_146, lora_1, LoRA_Strength)[0]

            if use_lora2 and lora_2 is not None:
                print("Loading LoRA 2...")
                unetloadergguf_146 = load_lora2.load_lora_model_only(unetloadergguf_146, lora_2, LoRA_Strength2)[0]

            if use_lora3 and lora_3 is not None:
                print("Loading LoRA 3...")
                unetloadergguf_146 = load_lora3.load_lora_model_only(unetloadergguf_146, lora_3, LoRA_Strength3)[0]

            cfgguider_135 = cfgguider.EXECUTE_NORMALIZED(
                cfg=1,
                model=unetloadergguf_146,
                positive=get_value_at_index(ltxvconditioning_111, 0),
                negative=get_value_at_index(ltxvconditioning_111, 1),
            )

            clear_output()

            print("First Sampling...")
            samplercustomadvanced_113 = samplercustomadvanced.EXECUTE_NORMALIZED(
                noise=get_value_at_index(randomnoise_133, 0),
                guider=get_value_at_index(cfgguider_135, 0),
                sampler=get_value_at_index(ksamplerselect_105, 0),
                sigmas=get_value_at_index(manualsigmas_134, 0),
                latent_image=get_value_at_index(ltxvconcatavlatent_121, 0),
            )

            # print("deleting cfg guider")
            del cfgguider_135
            torch.cuda.empty_cache()
            gc.collect()

            ltxvseparateavlatent_116 = ltxvseparateavlatent.EXECUTE_NORMALIZED(
                av_latent=get_value_at_index(samplercustomadvanced_113, 0)
            )

            ltxvcropguides_108 = ltxvcropguides.EXECUTE_NORMALIZED(
                positive=get_value_at_index(ltxvconditioning_111, 0),
                negative=get_value_at_index(ltxvconditioning_111, 1),
                latent=get_value_at_index(ltxvseparateavlatent_116, 0),
            )

            cfgguider_109 = cfgguider.EXECUTE_NORMALIZED(
                cfg=1,
                model=unetloadergguf_146,
                positive=get_value_at_index(ltxvcropguides_108, 0),
                negative=get_value_at_index(ltxvcropguides_108, 1),
            )

            # print("loading vae")
            vaeloader_144 = vaeloader.load_vae(vae_name=vae_model)

            latentupscalemodelloader = NODE_CLASS_MAPPINGS["LatentUpscaleModelLoader"]()
            latentupscalemodelloader_136 = latentupscalemodelloader.EXECUTE_NORMALIZED(
                model_name=upscaler_model
            )

            ltxvlatentupsampler_195 = ltxvlatentupsampler.upsample_latent(
                samples=get_value_at_index(ltxvcropguides_108, 2),
                upscale_model=get_value_at_index(latentupscalemodelloader_136, 0),
                vae=get_value_at_index(vaeloader_144, 0),
            )

            # print("deleting latent upscale model")
            del latentupscalemodelloader_136
            torch.cuda.empty_cache()
            gc.collect()


            ltxvimgtovideoinplace_130 = ltxvimgtovideoinplace.EXECUTE_NORMALIZED(
                strength=image_strength,
                bypass=image_bypass,
                vae=get_value_at_index(vaeloader_144, 0),
                image=get_value_at_index(ltxvpreprocess_126, 0),
                latent=get_value_at_index(ltxvlatentupsampler_195, 0),
            )

            # print("deleting vae")
            del vaeloader_144
            torch.cuda.empty_cache()
            gc.collect()


            if image_path is not None:
                ltxvconcatavlatent_129 = ltxvconcatavlatent.EXECUTE_NORMALIZED(
                    video_latent=get_value_at_index(ltxvimgtovideoinplace_130, 0),
                    audio_latent=get_value_at_index(ltxvseparateavlatent_116, 1),
                )
            else:
                ltxvconcatavlatent_129 = ltxvconcatavlatent.EXECUTE_NORMALIZED(
                    video_latent=get_value_at_index(ltxvlatentupsampler_195, 0),
                    audio_latent=get_value_at_index(ltxvseparateavlatent_116, 1),
                )


            print("Second Sampling...")
            samplercustomadvanced_119 = samplercustomadvanced.EXECUTE_NORMALIZED(
                noise=get_value_at_index(randomnoise_114, 0),
                guider=get_value_at_index(cfgguider_109, 0),
                sampler=get_value_at_index(ksamplerselect_106, 0),
                sigmas=get_value_at_index(manualsigmas_107, 0),
                latent_image=get_value_at_index(ltxvconcatavlatent_129, 0),
            )

            # print("deleting cfg guider")
            del cfgguider_109
            # print("deleting unet")
            del unetloadergguf_146
            torch.cuda.empty_cache()
            gc.collect()

            ltxvseparateavlatent_118 = ltxvseparateavlatent.EXECUTE_NORMALIZED(
                av_latent=get_value_at_index(samplercustomadvanced_119, 1)
            )

            # print("loading vae")
            vaeloader_144 = vaeloader.load_vae(vae_name=vae_model)

            vaedecode_117 = vaedecode.decode(
                samples=get_value_at_index(ltxvseparateavlatent_118, 0),
                vae=get_value_at_index(vaeloader_144, 0),
            )

            # print("deleting vae")
            del vaeloader_144
            torch.cuda.empty_cache()
            gc.collect()

            ltxvaudiovaedecode_120 = ltxvaudiovaedecode.EXECUTE_NORMALIZED(
                samples=get_value_at_index(ltxvseparateavlatent_118, 1),
                audio_vae=get_value_at_index(vaeloaderkj_142, 0),
            )

            # print("deleting audio vae")
            del vaeloaderkj_142
            torch.cuda.empty_cache()
            gc.collect()

            print("Creating Video...")
            createvideo_115 = createvideo.EXECUTE_NORMALIZED(
                fps=fpsv,
                images=get_value_at_index(vaedecode_117, 0),
                audio=get_value_at_index(ltxvaudiovaedecode_120, 0),
            )

            global output_path

            video = get_value_at_index(createvideo_115, 0)
            output_path = save_video_from_components(video)

            display_video(output_path)

image_file_1 = None
disable_image_for_Text_to_Video = False # @param {type:"boolean"}
prompt = "The lady says as she walks towards the camera, 'Hello my friend. Welcome to LTX2. You will enjoy creating videos with audio on LTX2.'" # @param {"type":"string"}
width = 480 # @param {"type":"number"}
height = 832 # @param {"type":"number"}
seed = 12 # @param {"type":"integer"}
frames = 121 # @param {"type":"integer", "min":1, "max":1200}
fps = 25 # @param {"type":"integer", "min":1, "max":60}

# @markdown ### LoRA Configuration
use_lora = False # @param {type:"boolean"}
LoRA_Strength = 1.0 # @param {"type":"slider","min":-100,"max":100,"step":0.01}
use_lora2 = False # @param {type:"boolean"}
LoRA_Strength2 = 1.0 # @param {"type":"slider","min":-100,"max":100,"step":0.01}
use_lora3 = False # @param {type:"boolean"}
LoRA_Strength3 = 1.0 # @param {"type":"slider","min":-100,"max":100,"step":0.01}

try:
    image_file_1 = file_uploaded
except NameError:
    pass

if disable_image_for_Text_to_Video:
    image_file_1 = None

mainLTX(
    image_path=image_file_1,
    prompt=prompt,
    width=width,
    height=height,
    seed=seed,
    frames=frames,
    fpsv=fps,
    use_lora=use_lora,
    LoRA_Strength=LoRA_Strength,
    use_lora2=use_lora2,
    LoRA_Strength2=LoRA_Strength2,
    use_lora3=use_lora3,
    LoRA_Strength3=LoRA_Strength3

)

First Sampling...


  0%|          | 0/8 [00:00<?, ?it/s]

Second Sampling...


  0%|          | 0/3 [00:00<?, ?it/s]

Creating Video...
